In [1]:
import numpy as np
import pandas as pd
import sqlite3
import gensim
import re
from nltk.corpus import stopwords
import nltk

In [2]:
# Use the NLTK downloader to download stopwords and punkt tokenizer (for breaking paragraphs into sentences)

#nltk.download()

In [3]:
!ls ../data

database.sqlite


In [4]:
sql_conn = sqlite3.connect('../data/database.sqlite')

In [5]:
mathematics = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'mathematics'",sql_conn)

computerscience = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'computerscience'",sql_conn)

history = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'history'",sql_conn)

philosophy = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'philosophy'",sql_conn)

elifive = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'explainlikeimfive'",sql_conn)

askanthro = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'AskAnthropology'",sql_conn)

homebrewing = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'Homebrewing'",sql_conn)

bicycling = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'bicycling'", sql_conn)

food = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'food'", sql_conn)

science = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'science'", sql_conn)

movies = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'movies'", sql_conn)

books = pd.read_sql("SELECT subreddit, body FROM May2015 WHERE subreddit == 'books'", sql_conn)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

In [9]:
# Array of tuples, with df and subject
subreddits = [(bicycling,'bicycling'),(history,'history'),(philosophy,'philosophy'),
              (elifive,'explain'),(homebrewing,'homebrew'),(askanthro,'anthropology'),
              (mathematics,'mathematics'),(computerscience,'computer science'),
              (food,'food'),(science,'science'),(movies,'movies'),(books,'books')]

In [10]:
for (subreddit,subject) in subreddits:
    print(subject+'\n')
    print(subreddit.info())
    print('=========\n')

bicycling

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40111 entries, 0 to 40110
Data columns (total 2 columns):
subreddit    40111 non-null object
body         40111 non-null object
dtypes: object(2)
memory usage: 940.1+ KB
None

history

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25242 entries, 0 to 25241
Data columns (total 2 columns):
subreddit    25242 non-null object
body         25242 non-null object
dtypes: object(2)
memory usage: 591.6+ KB
None

philosophy

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16943 entries, 0 to 16942
Data columns (total 2 columns):
subreddit    16943 non-null object
body         16943 non-null object
dtypes: object(2)
memory usage: 397.1+ KB
None

explain

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223148 entries, 0 to 223147
Data columns (total 2 columns):
subreddit    223148 non-null object
body         223148 non-null object
dtypes: object(2)
memory usage: 5.1+ MB
None

homebrew

<class 'pandas.core.frame.DataFrame'>
Int64

In [11]:
all_frames = [bicycling, history, philosophy, elifive, homebrewing, askanthro, mathematics,\
              computerscience, food, science, movies, books]
model_training_data = pd.concat(all_frames, ignore_index=True)

In [12]:
model_training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931315 entries, 0 to 931314
Data columns (total 2 columns):
subreddit    931315 non-null object
body         931315 non-null object
dtypes: object(2)
memory usage: 21.3+ MB


In [13]:
# Takes a sentence in a comment and converts it to a list of words.
def comment_to_wordlist(comment, remove_stopwords=False ):
    comment = re.sub("[^a-zA-Z]"," ", comment)
    words = comment.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return(words)

def comment_to_sentence(comment, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(comment.strip())
    
    sentences = []
    for s in raw_sentences:
        if len(s)>0:
            sentences.append(comment_to_wordlist(s, remove_stopwords))
    #rof
    return sentences

In [14]:
# Download a tokenizer to parse comments into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

comment_to_sentence(model_training_data.loc[1,'body'], tokenizer)

[[u'white', u'errywhere']]

In [15]:
sentences = []
for comment in model_training_data['body']:
    sentences += comment_to_sentence(comment, tokenizer)

In [16]:
print(len(sentences))

2477632


## Python Word2Vec parameters
Thanks to Kaggle for the great descriptions!

- **Architecture**: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
- **Training algorithm**: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
- **Downsampling of frequent words**: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
- **Word vector dimensionality**: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
- **Context / window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
- **Worker threads**: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
- **Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

In [18]:
num_features = 300
min_word_count = [10, 30, 50, 100]
context = [3, 5, 10]
downsampling = 1e-5
num_workers = 4

In [ ]:
from gensim.models import word2vec
import time
import logging
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num = 300

for mwc in min_word_count:
    for c in context:
        # Initialize and train each model 
        print("Training model with the following parameters:")
        message = "Size of feature vector: {}\nMin word count: {}\nn-gram context: {}\nDownsampling of frequent words: {}\n".format(num, mwc, c, downsampling)
        print(message)
        
        start = time.time();
        model = word2vec.Word2Vec(sentences, workers=num_workers, \
                    size=num, min_count = mwc, \
                    window = c, sample = downsampling)
        end = time.time();
        total = end-start;
        # Compress, name, and store each model
        model.init_sims(replace=True)
        model_name = str(num) + "features_" + str(mwc) + "minwords_" + str(c) + "context"
        print("Took " + str(total) + " time to train model: " + model_name)
        model.save(model_name)
        print("Model saved.")

Training model with the following parameters:
Size of feature vector: 300
Min word count: 10
n-gram context: 3
Downsampling of frequent words: 1e-05

Took 222.979872942 time to train model: 300features_10minwords_3context
Model saved.
Training model with the following parameters:
Size of feature vector: 300
Min word count: 10
n-gram context: 5
Downsampling of frequent words: 1e-05

Took 235.19748807 time to train model: 300features_10minwords_5context
Model saved.
Training model with the following parameters:
Size of feature vector: 300
Min word count: 10
n-gram context: 10
Downsampling of frequent words: 1e-05

Took 259.743896008 time to train model: 300features_10minwords_10context
Model saved.
Training model with the following parameters:
Size of feature vector: 300
Min word count: 30
n-gram context: 3
Downsampling of frequent words: 1e-05

Took 215.342049837 time to train model: 300features_30minwords_3context
Model saved.
Training model with the following parameters:
Size of featu

In [23]:
# This cell is used for training an individual model
from gensim.models import word2vec
import time
import logging

# Initialize and train each model 
print("Training model with the following parameters:")
message = "Size of feature vector: {}\nMin word count: {}\nn-gram context: {}\nDownsampling of frequent words: {}\n".format(300, 10, 3, downsampling)
print(message)
        
start = time.time();
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=300, min_count = 10, \
            window = 10, sample = downsampling)
end = time.time();
total = end-start;
# Compress, name, and store each model
model.init_sims(replace=True)
model_name = str(300) + "features_" + str(10) + "minwords_" + str(10) + "context"
print("Took " + str(total) + " time to train model: " + model_name)
model.save(model_name)
print("Model saved.")

Training model with the following parameters:
Size of feature vector: 300
Min word count: 10
n-gram context: 3
Downsampling of frequent words: 1e-05

Took 271.326533079 time to train model: 300features_10minwords_10context
Model saved.


### Cells from here down are a bit of testing/exploration with the models.

In [28]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [29]:
sum((model["man"]-model["child"])**2)

8.3183469452418421

In [30]:
# Not sure if this model is that great, I'm thinking the context window was too large
# Model 1:
#model.init_sims(replace=True)
#model_name = "300features_30minwords_15context_includescience"
#model.save(model_name)

# Can load the model later with Word2Vec.load()

In [31]:
training_frames = [bike_training, hist_training, phil_training, elif_training, brew_training]
training_data = pd.concat(training_frames, ignore_index=True)

In [32]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 24999
Data columns (total 2 columns):
subreddit    25000 non-null object
body         25000 non-null object
dtypes: object(2)
memory usage: 585.9+ KB


In [33]:
training_sentences = []
for comment in training_data['body']:
    training_sentences += comment_to_sentence(comment, tokenizer, True)

In [34]:
# Filter out sentences with less than 5 words, these are likely nonsensical
after_filter_training_sentences = filter(lambda x: len(x)>5, training_sentences)

In [35]:
print(len(training_sentences))
print(len(after_filter_training_sentences))

84710
49508


In [37]:
labels = ["bicycle","statistics","history","philosophy","homebrewing","anthropology","explain"]

def f(word,label):
    return abs(model.similarity(word, label))

def label(sentence,labels):
    # Initialize distance to be high
    best_distance = 1e8
    l = ""
    for label in labels:
        ssds = map(lambda x: f(x,label),sentence)
        #print("Label: " + label)
        #print(ssds)
        #average = sum(ssds)/len(ssds)
        if min(ssds) < best_distance:
            l = label
            best_distance = min(ssds)
    return l

test = after_filter_training_sentences[1]
print(test)
label(test,labels)

[u'm', u'sure', u'anyone', u'could', u'afford', u'bike', u'could', u'afford', u'pay', u'someone', u'clean', u'every', u'ride']


'anthropology'